In [33]:
import requests
import time
import json
from bs4 import BeautifulSoup
import webbrowser
import os
from IPython.core.display import display, HTML
from fake_useragent import UserAgent

In [34]:
page_url = 'https://www.livelib.ru/critique/feed-retsenzii-kritikov/~{}'
# HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [35]:
page_parsing = 1
card_parsing = 1
captha_try_number = 0
data = []

def create_session():
    session = requests.Session()
    session.headers.update({
        'Host': 'www.livelib.ru',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 YaBrowser/20.3.0.1223 Yowser/2.5 Yptp/1.56 Safari/537.36',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru,en;q=0.9',
        'Cookie': '__ll_fv=1584724821; __ll_tum=4122156853; __ll_ab_mp=1; __llutmz=-180; __llutmf=0; __ll_cp=1584724822; _ga=GA1.2.301004989.1584736858; _gid=GA1.2.1925018713.1584736858; _ym_d=1584736858; _ym_uid=1584736858680076911; __r=pc1828c1_; LiveLibId=9e4e5615cdcb46ae9a106539c05f985f; __ll_dvs=5; __ll_unreg_rt=0; __ll_unreg_r=1; iwatchyou=cf9f4f60c1786c7994191dae7e98f023; __ll_wtbr=1; __gr=g1127c3_g1120c4_g1217c5_g600c1_g537c1_g424c1_g1226c1_g426c2_g1240c2_g433c1_g1119c1_; llser=%1F%8B%08%00%00%00%00%00%00%03%95R%CDn%DB0%0C%7E%17%DFY%C7%8E%BD4%CA%5B%14%D8%D9%60%24%C6f%EDH%19%25%0Bp%8A%EE%D9G%1B%C96%60%3Bl%17%09%A2%F8%FDQB%F3j%3E%A2.%85%90c%21%9B%8AS4%FB%83%29J%2B%9C%F8%DBL%E5%85%C8%81P%8A%E4%EF%CC0%AE%F51%E4%F2%7B%B5_%9B%AB%DA%14%99%23%A7n%0A%7D%C7%AE8%B1%A9%AA%FAP%1D%9B%AA%DD%FD%D9p%DB%40%8D%29%AA%DD%F1%A5n%EB%97%C3%5E%B7%F6Ie%C3%EC%93%2C%9D%0D%8E%D6%9A%B2%BF%7D%DD.w%A6%E8%29XN%CBC%A5%DE%1D%0F%8F%8B%27j%95W%FFJ%7E%AC7%B4%82%3A%E5%122B%1A%26%A6%CE%A2%1D%94%19Mk%3E%D8l%06%1B%15%F9%95%F7%16b%2A%9B%B6%7D%3D%00%FA%14%3Cx%9Ax%08%F9%E7%0C%16%04%8F0z%EEgH%C8%3E%A6L%DE%A3%96%EF%03%DF%3D%F4%C2gm%9F%1DgN4%F9%85%C0%0E%B3%A3%88%10GYR%E8%03%84%04%FD%84w%B8%B2+H8%93%24%84%21%5C%84%87D%82%DB%10W%FF%AD%3A%8B%24%99-%95%82%CA%C6WN%16o%C9%0E%5BS%BD%0D%40S%FE%C5%7F%03%19%23O%FC%0EyB%A7%40%89%A3%1E%9EO%F9%7B%8C%8B%A0%1F%C9G%15%7F%F7%90%E1%8C%BDCG%80%C3%95%9C%DAFt%98y%15%DC%FF%8B%AB%E6%FF%3E%D1%A7f%F8b%0A%1F%1E%C1%E2%8D%BD%27%29Ng%1D%C2%E7%0F%25%B84%83%A4%02%00%00; __ll_dv=1584812692'
    })
    # session.headers.update({'User-Agent': UserAgent().random})
    return session

session = create_session()

In [50]:
page_parsing = 116

In [83]:
while True:
    r = session.get(page_url.format(page_parsing))
    soup = BeautifulSoup(r.content.decode('utf-8'), 'lxml')
        
    page_cards = soup.find_all('div', {'class': 'block-border card-block'})
    if len(page_cards) == 0:
        if captha_try_number < 3:
            with open('captha.html', 'w', encoding='utf-8') as file:
                file.write(str(soup))
            # webbrowser.open('file://' + os.path.realpath('captha.html'))
            
            print('[!!!] | No page cards found, sleeping and restarting...', end='')
            seesion = create_session()
            time.sleep(5)
            print(' Done.')
            captha_try_number += 1
        else:
            print('[xxx] | Seems no new pages left.')
            break
    
    for div_card in page_cards:
        captha_try_number = 0
        _tstart = time.time()
        
        div_content = div_card.find('div', {'class': 'group-review'})
        book_img = div_content.findNext('img').attrs['data-pagespeed-lazy-src']
        
        book_raw_info = div_content.findNext('font')
        if book_raw_info is None:
            book_raw_info = div_content.findNext('div', {'class': 'scifi-review-author'})
        if book_raw_info is None:
            # Unknown rare format
            print(f'[{round(time.time() - _tstart, 1)}] | {page_parsing}#{str(card_parsing).zfill(4)} | {book_title} - SKIPPED UNKNOWN FORMAT')
            card_parsing += 1
            continue
            
        book_raw_info = book_raw_info.text.split('\n')
            
        book_reviewer = book_raw_info[0].split(':')[1].strip()
        book_title = book_raw_info[1][18:]
        
        book_review_url = 'https://www.livelib.ru' + div_content.findNext('a').findNext('a').attrs['href']
        r_review = session.get(book_review_url)
        rsoup = BeautifulSoup(r_review.content.decode('utf-8'), 'lxml')
        
        breview = rsoup.find('div', {'class': 'backgr-block post'})
        breview_text = ' '.join([tag.text.strip() for tag in breview.find_all('p')[1:]])
        
        try:
            book_url = 'https://www.livelib.ru' + div_content.findNext('font').findNext('a').attrs['href']
        except AttributeError:
            book_url = 'https://www.livelib.ru' + div_content.find('div', {'class': 'scifi-review-author'}).findNext('a').attrs['href']
        book_r = session.get(book_url)
        bsoup = BeautifulSoup(book_r.content.decode('utf-8'), 'lxml')
        
        book_author = bsoup.find('a', {'id': 'book-author'})
        if book_author is None:
            book_author = bsoup.find('span', {'id': 'book-author'})
        if book_author is None:
            # No author on page
            book_author = 'Undefined'
        else:
            book_author = book_author.text.replace('\n', '').strip()
        
        data.append([book_title, book_author, book_url, book_img, book_reviewer, breview_text])
        print(f'[{round(time.time() - _tstart, 1)}] | {page_parsing}#{str(card_parsing).zfill(4)} | {book_title}, {book_author} - {breview_text[:40]}...')
        card_parsing += 1
    
    
    if len(page_cards) != 0:
        jsonfilename = str(page_parsing).zfill(3)
        with open(f'data/page_{jsonfilename}.json', 'w', encoding='utf-8') as file:
            json.dump({f'{jsonfilename}': data}, file, ensure_ascii=False, indent=2)
            print(f'[0.0] Scraped data from page {page_parsing} saved to page_{str(page_parsing).zfill(3)}.json')
            data.clear()
    
        page_parsing += 1

[0.9] | 129#0149 | Мышиная Гвардия. Осень 1152, Undefined - В 1997 году Дэвид Питерсен написал на са...
[0.9] | 129#0150 | Не могу остановиться, Undefined - Ведь научный прогресс есть? Может быть, ...
[0.9] | 129#0151 | Не прощаюсь, Undefined - Во-первых (и это, пожалуй, самое важное)...
[0.8] | 129#0152 | Что они несли с собой, Undefined - В других рассказах повествование превращ...
[0.8] | 129#0153 | Тобол. Мало избранных, Undefined - Черт свое слово сдержал. Из-под пера Ива...
[0.8] | 129#0154 | Петровы в гриппе и вокруг него, Undefined - Так вот, такой успех — ​это пять тысяч (...
[0.8] | 129#0155 | Пробуждение, Undefined - Была одна вдова разорившегося плантатора...
[1.0] | 129#0156 | Не прощаюсь, Undefined - С этого начинается путешествие по постре...
[0.9] | 129#0157 | Спящие гиганты, Undefined - Гигантские боевые роботы всегда были час...
[0.0] | 129#0158 | Спящие гиганты - SKIPPED UNKNOWN FORMAT
[0.0] Scraped data from page 129 saved to page_129.json
[0.6] | 130#0159 | Мир, ко

[0.7] | 137#0229 | Девочки, Undefined - Главная героиня Эви, угодившая на ранчо ...
[0.7] | 137#0230 | Дым, Undefined - Идею этого романа Дэну Вилете подсказал ...
[0.7] | 137#0231 | Революция от первого лица, Undefined - В западной науке сталинский режим долго ...
[1.0] | 137#0232 | Зеленый. История цвета, Undefined - У древних греков в языке не было специал...
[1.0] | 137#0233 | Дьявол в Белом городе, Undefined - «Дьявол в Белом городе» — это не один сю...
[0.9] | 137#0234 | Отдай мое сердце, Undefined - «Сновидения Ехо», долгожданный новый цик...
[1.0] | 137#0235 | Главарь банды на день, Undefined - Дабнер и Левитт описывают Венкатеша эдак...
[0.9] | 137#0236 | Семиевие, Undefined - Космическая фантастика от Нила Стивенсон...
[0.9] | 137#0237 | Сатанинское танго, Undefined - Когда в 2015 году венгерский писатель Ла...
[1.1] | 137#0238 | Пожарный, Undefined - Как известно, уцелеть в мире Стивена Кин...
[0.0] Scraped data from page 137 saved to page_137.json
[1.1] | 138#0239 | Задача 

[0.6] | 145#0312 | Ничейного отца дети, Undefined - Роман «Мастер снов», давший начало однои...
[0.6] | 145#0313 | Ничейного отца дети, Undefined - У такой многогранности есть оборотная ст...
[0.7] | 145#0314 | Ничейного отца дети, Undefined - В «Катушке синих ниток» она задавалась в...
[0.6] | 145#0315 | Ничейного отца дети, Undefined - Надо сказать, что на фоне сюжетной аскез...
[0.6] | 145#0316 | Ничейного отца дети, Undefined - «Лучшее — враг хорошего». «Лучше меньше,...
[0.5] | 145#0317 | Черный ветер, белый снег, Undefined - И все это при том, что Кловер практикует...
[0.6] | 145#0318 | iPhuck 10, Undefined - Не в том смысле рады, что, мол, еще одна...
[0.0] Scraped data from page 145 saved to page_145.json
[0.6] | 146#0319 | Страшная тайна, Undefined - Вселенная, описанная в цикле «Магиды», п...
[0.8] | 146#0320 | Город Брежнев, Undefined - ...
[0.7] | 146#0321 | Люди и собаки, Undefined - ...
[0.6] | 146#0322 | iPhuck 10, Undefined - Год назад мы с осторожным оптимизмом пре...


[0.6] | 154#0398 | Острые края, Undefined - Когда Джо Аберкромби выпустил в свет пер...
[0.6] | 154#0399 | Голоса чертовски тонки, Undefined - Записать Уильяма нашего Шекспира в фанта...
[0.6] | 154#0400 | Белгравия, Undefined - По сравнению с «Аббатством» Феллоуз сдви...
[0.6] | 154#0401 | Его кровавый проект, Undefined - В 1869 году шотландский Хайленд (тот сам...
[0.6] | 154#0402 | Обмани-Смерть, Undefined - ...
[0.6] | 154#0403 | Иуда, Undefined - Начало более чем обескураживающее, потом...
[0.7] | 154#0404 | Любовь гика, Undefined - Это книга, породившая культ. Американка ...
[0.6] | 154#0405 | Чудеса и фантазии, Undefined - Лауреат «Букера» и титулованная Дама (бр...
[0.8] | 154#0406 | Свет мой, зеркальце, Undefined - «Свет мой зеркальце, скажи, да всю правд...
[0.8] | 154#0407 | Квантовый вор, Undefined - «Квантовый вор» цепляет с первых же стро...
[0.0] Scraped data from page 154 saved to page_154.json
[0.7] | 155#0408 | Эрос и магия в эпоху Возрождения, Undefined - Дебютный ро

[0.6] | 162#0480 | Синий взгляд Смерти. Рассвет. Часть первая, Undefined - На протяжении большей части своей писате...
[0.6] | 162#0481 | Останется при мне, Undefined - Главный герой и рассказчик, пожилой проф...
[0.6] | 162#0482 | Вот идет человек: роман-автобиография, Undefined - Начинается история Сайки Гронаха — не то...
[0.6] | 162#0483 | Красный Крест, Undefined - Позиция писателя предельно ясна – он кат...
[0.6] | 162#0484 | Недоверчивые умы. Чем нас привлекают теории заговоров, Undefined - ...
[0.5] | 162#0485 | Money. Неофициальная биография денег, Undefined - ...
[0.6] | 162#0486 | Хочу и буду. Принять себя, полюбить жизнь и стать счастливым, Undefined - ...
[0.5] | 162#0487 | Гибкое сознание. Новый взгляд на психологию развития взрослых и детей, Undefined - ...
[0.0] Scraped data from page 162 saved to page_162.json
[0.5] | 163#0488 | Свое время, Undefined - Сюжетный конфликт построен на вояже пред...
[0.7] | 163#0489 | Номер 11, Undefined - Уже тридцать лет Коу нещадно крит

...

[0.7] | 194#0799 | Дальнее чтение, Undefined - ...
[0.6] | 194#0800 | Красный Марс, Undefined - Марсианская трилогия Кима Стэнли Робинсо...
[0.6] | 194#0801 | Вилы, Undefined - Пугачевский бунт появился в творчестве И...
[0.6] | 194#0802 | Тайна лабиринта. Как была прочитана забытая письменность, Undefined - Тема, за которую берется Маргалит Фокс, ...
[0.7] | 194#0803 | С ума сойти! Путеводитель по психическим расстройствам для жителя большого города, Undefined - Каждое заболевание — отдельная глава. На...
[0.7] | 194#0804 | Дом на хвосте паровоза: путеводитель по Европе в сказках Андерсена, Undefined - Каждая глава в книге Горбунова имеет наз...
[0.6] | 194#0805 | Предрассветная лихорадка, Undefined - Действие книги начинается на корабле, пл...
[0.7] | 194#0806 | Боги среди людей, Undefined - Те, кто читал «Жизнь после жизни», отлич...
[0.6] | 194#0807 | "Сглотнула рыба их…". Беседы о счастье, Undefined - Откровенно говоря, всё, что может быть о...
[0.0] Scraped data from page 194 sav

[0.8] | 202#0881 | Сказания Меекханского пограничья: Север - Юг, Undefined - В глазах отечественного читателя безусло...
[0.7] | 202#0882 | Лучшее из Майлза, Undefined - ...
[0.7] | 202#0883 | Кваzи, Undefined - Мэтр российской фантастики Сергей Лукьян...
[0.8] | 202#0884 | Триумф красной герани. Книга о Будапеште, Undefined - ...
[0.8] | 202#0885 | Леопард за стеклом, Undefined - В Греции происходит примерно то же, что ...
[0.7] | 202#0886 | Рассказы про Франца и футбол, Undefined - Итак, в 13-й части рассказывается о тако...
[0.9] | 202#0887 | Красный ошейник, Undefined - Каждый день к Морлаку в камеру ходит сле...
[0.0] Scraped data from page 202 saved to page_202.json
[0.6] | 203#0888 | Мои странные мысли, Undefined - Памук любит сложную фабулу, и в «Моих ст...
[0.6] | 203#0889 | Проповедь о падении Рима, Undefined - «Проповедь о падении Рима» несмотря на н...
[0.5] | 203#0890 | 13-й апостол. Маяковский. Трагедия-буфф в шести действиях, Undefined - ...
[0.7] | 203#0891 | Аквамен. К

[1.1] | 210#0963 | Небесный Стокгольм, Undefined - Что все политические анекдоты сочинялись...
[0.8] | 210#0964 | Вляпалась!, Undefined - С начала XX века «неформатные» англо-аме...
[0.7] | 210#0965 | Дикий Лебедь и другие сказки, Undefined - Сказки у Каннингема не перекликаются меж...
[0.7] | 210#0966 | Нежность к мертвым, Undefined - ...
[0.0] Scraped data from page 210 saved to page_210.json
[0.7] | 211#0967 | Чертополох. Философия живописи, Undefined - ...
[0.8] | 211#0968 | Чертополох. Философия живописи, Undefined - ...
[0.7] | 211#0969 | Чертополох. Философия живописи, Undefined - Книга Сергея Кузнецова полна описаниями ...
[0.8] | 211#0970 | Чертополох. Философия живописи, Undefined - Однажды дождливой ночью в Майами тренер ...
[0.8] | 211#0971 | Чертополох. Философия живописи, Undefined - Уитшенки — любящие и преданные друг друг...
[0.6] | 211#0972 | Чертополох. Философия живописи, Undefined - ...
[0.7] | 211#0973 | Чертополох. Философия живописи, Undefined - Как ни странно, с

[0.7] | 218#1046 | В интернете кто-то неправ! Научные исследования спорных вопросов, Undefined - Очевидно, что нынешняя культурная ситуац...
[0.0] Scraped data from page 218 saved to page_218.json
[0.7] | 219#1047 | Семь жизней, Undefined - ...
[0.8] | 219#1048 | Бразилья, Undefined - Существует бессчётное множество книг, гд...
[0.9] | 219#1049 | Непохожие поэты, Undefined - Друзья (как быть?), не любят стихотворце...
[0.6] | 219#1050 | Авиатор, Undefined - Абстрактное «читатель» можно в данном сл...
[0.5] | 219#1051 | Травля, Undefined - ...
[0.9] | 219#1052 | Чтение. Письмо. Эссе о литературе, Undefined - ...
[0.7] | 219#1053 | Вдовий плат. Знак Каина, Undefined - ...
[0.7] | 219#1054 | Лавка дурных снов, Undefined - В первую очередь Кинг – мастер крупной ф...
[0.9] | 219#1055 | Москва и Петербург - как мы их не знаем, Undefined - Это первый сдвоенный том антологии совре...
[0.7] | 219#1056 | Семь жизней, Undefined - ...
[0.0] Scraped data from page 219 saved to page_219.json
[0.7] |

[0.6] | 227#1132 | Шукшин, Undefined - Варламов ставит во главу угла — того угл...
[0.6] | 227#1133 | Диккенс, Undefined - Другое дело, что после Пирсона и Томалин...
[0.7] | 227#1134 | Маленький друг, Undefined - Сложнее всего рассказывать о том, что по...
[0.5] | 227#1135 | Детство Иисуса, Undefined - Автор как будто ведет игру с читателем, ...
[0.6] | 227#1136 | Коротким будет приговор, Undefined - «Почти вся Европа» работала на оккупанто...
[0.0] Scraped data from page 227 saved to page_227.json
[0.5] | 228#1137 | Острова утопии: Педагогическое и социальное проектирование послевоенной школы (1940—1980-е), Undefined - Есть стойкая легенда, будто генсек Андро...
[0.6] | 228#1138 | Детство Иисуса, Undefined - Вообще, совсем не так много на свете кни...
[0.5] | 228#1139 | Инноваторы. Как несколько гениев, хакеров и гиков совершили цифровую революцию, Undefined - Первая героиня Айзексона — дочь Байрона ...
[0.6] | 228#1140 | Эволюция разума, Undefined - Используя физиологию мозга в каче

In [62]:
r = session.get(page_url.format(page_parsing))
soup = BeautifulSoup(r.content.decode('utf-8'), 'lxml')

In [63]:
page_cards = soup.find_all('div', {'class': 'block-border card-block'})
div_card = page_cards[5]

In [77]:
div_content = div_card.find('div', {'class': 'group-review'})
div_content

<div class="group-review">
<div class="with-pad event-header block-border-b uerow"><div class="rel"><span class="right">
</span>
<div class="group-user-date" style="margin-bottom:0px;line-height:20px;">
<div class="login-hide">
<a href="/reader/Katerina" title="Katerina"><div class="event-user-avatar big" style="background-image: url(https://i.livelib.ru/userpic/Katerina/40x40/fbd8/Katerina.jpg);"></div></a>
<div class="group-login-date">
<span class="date-topright">
<a href="/critique/post/34326-natalya-kochetkova-o-knige-doklad-o-meduzah"><span class="date">31 марта 2018 г., 17:21</span></a>
</span>
<a class="a-login-black" href="/reader/Katerina" title="Katerina">Katerina</a>
                                                    
                                                            создала обсуждение<br/>
<a href="/critique/post/34326-natalya-kochetkova-o-knige-doklad-o-meduzah" title="Наталья Кочеткова о книге «Доклад о медузах»">Наталья Кочеткова о книге «Доклад о медузах»</a

In [67]:
book_img = div_content.findNext('img').attrs['data-pagespeed-lazy-src']
book_img

'https://u.livelib.ru/album/1000004802/s/5xspq8x2/o-s.jpg'

In [68]:
div_content

<div class="group-review">
<div class="with-pad event-header block-border-b uerow"><div class="rel"><span class="right">
</span>
<div class="group-user-date" style="margin-bottom:0px;line-height:20px;">
<div class="login-hide">
<a href="/reader/Katerina" title="Katerina"><div class="event-user-avatar big" style="background-image: url(https://i.livelib.ru/userpic/Katerina/40x40/fbd8/Katerina.jpg);"></div></a>
<div class="group-login-date">
<span class="date-topright">
<a href="/critique/post/34326-natalya-kochetkova-o-knige-doklad-o-meduzah"><span class="date">31 марта 2018 г., 17:21</span></a>
</span>
<a class="a-login-black" href="/reader/Katerina" title="Katerina">Katerina</a>
                                                    
                                                            создала обсуждение<br/>
<a href="/critique/post/34326-natalya-kochetkova-o-knige-doklad-o-meduzah" title="Наталья Кочеткова о книге «Доклад о медузах»">Наталья Кочеткова о книге «Доклад о медузах»</a

In [70]:
book_url = 'https://www.livelib.ru' + div_content.findNext('font').findNext('a').attrs['href']
book_r = session.get(book_url)
bsoup = BeautifulSoup(book_r.content.decode('utf-8'), 'lxml')

AttributeError: 'NoneType' object has no attribute 'findNext'

In [80]:
div_content.find('div', {'class': 'scifi-review-author'}).findNext('a').attrs['href']

'/book/1002712241'

In [55]:
book_raw_info = div_content.findNext('font').text.split('\n')
book_reviewer = book_raw_info[0].split(':')[1].strip()
book_title = book_raw_info[1][18:]
book_raw_info, book_reviewer, book_title

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
book_url = 'https://www.livelib.ru' + div_content.findNext('font').findNext('a').attrs['href']
book_url

In [ ]:
book_review_url = 'https://www.livelib.ru' + div_content.findNext('a').findNext('a').attrs['href']
book_review_url

In [ ]:
r_review = requests.get(book_review_url, headers=HEADERS)
rsoup = BeautifulSoup(r_review.content.decode('utf-8'), 'lxml')

In [ ]:
rsoup

In [ ]:
breview = rsoup.find('div', {'class': 'backgr-block post'})
breview

In [ ]:
b_texts = ' '.join([tag.text.strip() for tag in breview.find_all('p')[1:-1]])
b_texts

In [ ]:
book_r = session.get(book_url)
bsoup = BeautifulSoup(book_r.content.decode('utf-8'), 'lxml')

In [ ]:
bsoup.find('a', {'id': 'book-author'}).text

In [ ]:
# Double author
book_url = 'https://www.livelib.ru/book/1003491630'

book_r = session.get(book_url)
bsoup = BeautifulSoup(book_r.content.decode('utf-8'), 'lxml')

In [ ]:
try:
    book_author = bsoup.find('a', {'id': 'book-author'}).text
except:
    span_authors = bsoup.find('span', {'id': 'book-author'}).text.replace('\n', '')

span_authors

In [ ]:
book_url = 'https://www.livelib.ru/book/1003008118'
book_r = session.get(book_url)
bsoup = BeautifulSoup(book_r.content.decode('utf-8'), 'lxml')

In [ ]:
book_author = bsoup.find('a', {'id': 'book-author'})
if book_author is None:
    book_author = bsoup.find('span', {'id': 'book-author'})
if book_author is None:
    # No author on page
    book_author = 'Undefined'
else:
    book_author = book_author.text.replace('\n', '').strip()

book_author

In [ ]:
bsoup